In [ ]:
# Necessary Packages 
from bs4 import BeautifulSoup
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd

# Create a ChromeOptions object to modify the browser settings
chrome_options = Options()

# Disable location access (block websites from accessing location)
prefs = {
    "profile.default_content_setting_values.geolocation": 2  # 2 means block
}
chrome_options.add_experimental_option("prefs", prefs)

# Initialize the Chrome WebDriver with the modified options
driver = webdriver.Chrome(options=chrome_options)

# Function to get job links from multiple pages
def get_job_links(base_url, page_start, page_end):
    job_links_list = []
    for page in range(page_start, page_end + 1):
        url = f"{base_url}tat-ca-viec-lam-trang-{page}-vi.html"
        driver.get(url)
        time.sleep(random.randint(5, 10))
        for job in driver.find_elements(By.CSS_SELECTOR, 'h2 > .job_link'):
            job_links_list.append(job.get_attribute('href'))

    return job_links_list

# Function to crawl job information for each job link
def crawl_job_information(job_urls):
    job_information_list = []

    for job_url in job_urls:
        driver.get(job_url)
        time.sleep(random.randint(2, 5))  # Wait for the page to load
        
        try:
            # Lấy tên công việc
            try:
                job_name = driver.find_element(By.CSS_SELECTOR, 'h1.title').text.strip()
            except:
                job_name = "Not specified"

            # Lấy thông tin lương
            try:
                salary_info = driver.find_element(By.XPATH, "//strong[contains(text(),'Lương')]/following-sibling::p").text.strip()
            except:
                salary_info = "Not specified"

            # Lấy địa điểm
            try:
                location_elements = driver.find_elements(By.CSS_SELECTOR, '.map p a')
                locations = ', '.join([loc.text.strip() for loc in location_elements]) if location_elements else "Not specified"
            except:
                locations = "Not specified"
            
            # Lấy kinh nghiệm
            try:
                experience = driver.find_element(By.XPATH, "//strong[contains(text(),'Kinh nghiệm')]/following-sibling::p").text.strip()
            except:
                experience = "Not specified"
            
            # Lấy cấp bậc
            try:
                job_level = driver.find_element(By.XPATH, "//strong[contains(text(),'Cấp bậc')]/following-sibling::p").text.strip()
            except:
                job_level = "Not specified"
            
            # Lấy ngày hết hạn nộp hồ sơ
            try:
                deadline = driver.find_element(By.XPATH, "//strong[contains(text(),'Hết hạn nộp')]/following-sibling::p").text.strip()
            except:
                deadline = "Not specified"
            
            # Lấy ngày cập nhật
            try:
                update_date = driver.find_element(By.XPATH, "//strong[contains(text(),'Ngày cập nhật')]/following-sibling::p").text.strip()
            except:
                update_date = "Not specified"
            
            # Lấy loại hình công việc
            try:
                employment_type = driver.find_element(By.XPATH, "//strong[contains(text(),'Hình thức')]/following-sibling::p").text.strip()
            except:
                employment_type = "Not specified"
            
            # Lấy loại nghành nghề  
            try:
                li_briefcase_element = driver.find_element(By.CSS_SELECTOR,"li:has(.mdi-briefcase)")
                a_briefcase_elements=li_briefcase_element.find_elements(By.CSS_SELECTOR,"a")
                list_briefcase=""
                for link in a_briefcase_elements:
                    list_briefcase+=link.text+", "   
                list_briefcase=list_briefcase.rstrip(", ")
            except:
                list_briefcase="Not Found"
                
            # Lấy danh sách phúc lợi
            try:
                welfare_elements = driver.find_elements(By.CSS_SELECTOR, "ul.welfare-list li")
                welfare_list = ', '.join([welfare.text.strip() for welfare in welfare_elements if welfare.text.strip()])
            except:
                welfare_list = "Not specified"
            
            # Lấy phần "Mô tả Công việc"
            try:
                # Tìm tất cả các phần tử có liên quan tới "Mô tả Công việc"
                job_description_section = driver.find_elements(By.XPATH, "//h2[contains(text(),'Mô tả Công việc')]/following-sibling::*")
                
                job_description_list = []
                
                # Lặp qua tất cả các phần tử sau tiêu đề
                for section in job_description_section:
                    if section.tag_name == "p":
                        # Nếu phần tử là <p>, lấy văn bản nếu có chứa thông tin
                        if section.text.strip():
                            job_description_list.append(section.text.strip())
                    elif section.tag_name == "ul":
                        # Nếu phần tử là <ul>, lấy các phần tử con <li>
                        description_items = section.find_elements(By.TAG_NAME, "li")
                        job_description_list.extend([item.text.strip() for item in description_items if item.text.strip()])
                
                # Gộp tất cả các đoạn mô tả lại thành một chuỗi
                job_description = '\n'.join(job_description_list)

            except Exception as e:
                job_description = "Not specified"

            # Lấy phần "Yêu Cầu Công Việc"
            try:
                job_requirements_section = driver.find_elements(By.XPATH, "//h2[contains(text(),'Yêu Cầu Công Việc')]/following-sibling::*")
                
                job_requirements_list = []
                
                # Lặp qua tất cả các phần tử sau tiêu đề
                for section in job_requirements_section:
                    if section.tag_name == "p":
                        # Nếu phần tử là <p>, lấy văn bản nếu có chứa thông tin
                        if section.text.strip():
                            job_requirements_list.append(section.text.strip())
                    elif section.tag_name == "ul":
                        # Nếu phần tử là <ul>, lấy các phần tử con <li>
                        requirements_items = section.find_elements(By.TAG_NAME, "li")
                        job_requirements_list.extend([item.text.strip() for item in requirements_items if item.text.strip()])
                
                # Gộp tất cả các đoạn yêu cầu lại thành một chuỗi
                job_requirements = '\n'.join(job_requirements_list)

            except Exception as e:
                job_requirements = "Not specified"
            
            # Lấy nội dung của phần "Thông tin khác"
            try:
                other_info_section = driver.find_element(By.XPATH, "//h3[contains(text(),'Thông tin khác')]/following-sibling::div/ul")
                other_info_items = other_info_section.find_elements(By.TAG_NAME, "li")
                other_info = '\n'.join([item.text.strip() for item in other_info_items if item.text.strip()])
            except:
                other_info = "Not specified"
                
            # Lấy danh sách các Job Tags / Skills
            try:
                # Tìm tất cả các phần tử <li> trong phần "Job Tags / Skills"
                job_tags_elements = driver.find_elements(By.CSS_SELECTOR, "div.job-tags ul li")
                
                # Lấy text của từng mục <li> và ghép lại thành chuỗi, ngăn cách bằng dấu phẩy
                job_tags = ', '.join([tag.text.strip() for tag in job_tags_elements if tag.text.strip()])
            except:
                job_tags = "Not specified"
                
            # Lấy danh sách các link công việc tương tự
            try:
                similar_job_links = driver.find_elements(By.CSS_SELECTOR, "div.jobs-list a.job_link")
                similar_jobs = [link.get_attribute("href") for link in similar_job_links if link.get_attribute("href")]
            except:
                similar_jobs = []
                
            # Tìm và nhấp vào "Tổng quan công ty"
            try:
                company_overview_link = driver.find_element(By.XPATH, "//a[contains(text(), 'Tổng quan công ty')]")
                company_overview_link.click()
                time.sleep(3)  # Đợi trang công ty tải
            except:
                print("Không tìm thấy link 'Tổng quan công ty'.")
            # Tìm và nhấp vào tên công ty để vào trang chi tiết của công ty
            try:
                company_name_link = driver.find_element(By.CSS_SELECTOR, "a.name")
                company_name_link.click()
                time.sleep(3)  # Đợi trang chi tiết công ty tải
            except:
                print("Không tìm thấy link tên công ty.")

            # Lấy tên công ty
            try:
                company_name = driver.find_element(By.XPATH, "//h1[contains(@class,'name')]").text.strip()
            except:
                company_name = "Not specified"

            # Lấy địa điểm công ty
            try:
                company_address = driver.find_element(By.XPATH, "//strong[contains(text(),'Địa điểm')]/following-sibling::p").text.strip()
            except:
                company_address = "Not specified"

            # Lấy quy mô công ty
            try:
                company_size_element = driver.find_element(By.XPATH, "//li[span[contains(@class, 'mdi-account-supervisor')]]")
                company_size_text = company_size_element.text
                company_size = company_size_text.split("Quy mô công ty:")[-1].strip()  # Chỉ lấy phần sau "Quy mô công ty:"
            except:
                company_size = "Not specified"

            # Lấy loại hình hoạt động
            try:
                company_type_element = driver.find_element(By.CSS_SELECTOR, "li:has(.mdi-gavel)")
                company_type = company_type_element.text.replace("Loại hình hoạt động: ", "").strip()
            except:
                company_type = "Not specified"

            # Lấy đường dẫn website công ty
            try:
                company_website_element = driver.find_element(By.CSS_SELECTOR, "li:has(.mdi-link)")
                company_website = company_website_element.text.replace("Website: ", "").strip()
            except:
                company_website = "Not specified"

            # Lấy giới thiệu công ty
            try:
                company_intro_element = driver.find_element(By.CSS_SELECTOR, ".intro-section .main-text")
                company_intro = company_intro_element.text.strip()
            except:
                company_intro = "Not specified"

            # Thêm vào danh sách kết quả
            job_information = {
                "Job Name": job_name,
                "Salary": salary_info,
                "Location": locations,
                "Experience": experience,
                "Job Level": job_level,
                "Deadline": deadline,
                "Update Date": update_date,
                "Employment Type": employment_type,
                "Industry": list_briefcase,
                "Welfare": welfare_list,
                "Job Description": job_description,
                "Job Requirements": job_requirements,
                "Other Information": other_info,
                "Job Tags": job_tags,
                "Similar Jobs": similar_jobs,
                "Company Name": company_name,
                "Company Address": company_address,
                "Company Scale": company_size,
                "Company Type": company_type,
                "Company Website": company_website,
                "Company Introduction": company_intro,
                "Job Link": job_url
            }
            job_information_list.append(job_information)

        except Exception as e:
            print(f"Error processing job at '{job_url}': {e}")

    return pd.DataFrame(job_information_list)

# Base URL for job listing pages
BASE_URL = "https://careerviet.vn/viec-lam/"

# Get job links from multiple pages (e.g., pages from 1 to 3)
job_links = get_job_links(BASE_URL, 1, 3)

# Crawl job information based on collected job links
job_df = crawl_job_information(job_links)

# Print the DataFrame
print(job_df)

# Đóng trình duyệt sau khi hoàn tất
driver.quit()


Không tìm thấy link 'Tổng quan công ty'.
Không tìm thấy link 'Tổng quan công ty'.
Không tìm thấy link tên công ty.
Không tìm thấy link tên công ty.
Không tìm thấy link tên công ty.
Không tìm thấy link tên công ty.
Không tìm thấy link 'Tổng quan công ty'.
Không tìm thấy link tên công ty.
Không tìm thấy link tên công ty.
Không tìm thấy link 'Tổng quan công ty'.
Không tìm thấy link tên công ty.
Không tìm thấy link tên công ty.
Không tìm thấy link 'Tổng quan công ty'.
Không tìm thấy link tên công ty.
Không tìm thấy link tên công ty.
Không tìm thấy link tên công ty.
                                              Job Name             Salary  \
0                                        Not specified      Not specified   
1                      Chuyên Viên Phân Tích Tài Chính         Cạnh tranh   
2                            Nhân viên Brand Marketing   9 Tr - 15 Tr VND   
3     Financial & Investment Assistant to the Chairman         Cạnh tranh   
4    Trưởng Phòng Kế Hoạch Tài Chính (Finance M

In [15]:
job_df.to_csv('job_data.csv', index=False, encoding='utf-8-sig')